In [1]:
import xmlrpc.client
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import datetime
import os
import pandas as pd


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)
server = SimpleXMLRPCServer(("127.0.0.1", 8018),
                            requestHandler=RequestHandler)

# Настройка максимального размера лога
max_log_size = 10
current_log_size = None
# Данные файла лога
log_path = 'log/'

# Добавление строки в лог
def add_log(sname, work_time):
    global log_path
    global current_log_size
    if current_log_size is None:
        f = open(log_path + 'log.csv','w+')
        current_log_size = len(f.read().split('\n')) - 1
        f.close()
    f = open(log_path + 'log.csv','a')
    f.write(sname + ',' + datetime.datetime.now().strftime('%Y%m%dT%H:%M:%S') + ',' + str(work_time) + '\n')
    f.close()
    current_log_size += 1
    check_log_size()
    return True
server.register_function(add_log, 'add_log')

# Проверка размера лога, сохранение лога в отдельный файл
def check_log_size():
    global log_path
    global max_log_size
    global current_log_size
    if current_log_size == max_log_size:
        os.rename(log_path + 'log.csv', log_path + 'log_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv')
        current_log_size = 0

def get_logs_slice(p_type, s_time, e_time, max_work_time):
    global log_path
    f = open(log_path + 'log.csv','r')
    text = f.read().split('\n')
    logs = []
    for line in text:
        if line != '':
            log = line.split(',')
            if (p_type == 'all' or log[0] == p_type) and (s_time == '' or log[1] >= s_time) and (e_time == '' or log[1] <= e_time) and (max_work_time == '' or float(log[2]) <= max_work_time):
                logs.append((log[0], log[1], log[2]))
    f.close()
    return logs        
server.register_function(get_logs_slice, 'get_logs_slice')

print ("Listening on port 8018...")
server.serve_forever()

Listening on port 8018...


127.0.0.1 - - [19/Dec/2024 19:46:09] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 19:46:09] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 